In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split,learning_curve,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score,roc_curve,confusion_matrix
from scipy import stats

In [7]:
data_train = pd.read_csv('train_reprocess.csv',encoding='big5')
data_test = pd.read_csv('test_reprocess.csv',encoding='big5')
submit_test = pd.read_csv('submit_test.csv',encoding='big5')

In [8]:
#stdc = StandardScaler()
X,y = data_train.drop(['Y1','CUS_ID'],axis=1), data_train['Y1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)
#X_train_std = stdc.fit_transform(X_train)
#X_test_std = stdc.fit_transform(X_test)

## XGBoost

In [9]:
from xgboost import XGBClassifier
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.2, max_delta_step=10,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=1,
       reg_alpha=10, reg_lambda=5, scale_pos_weight=0.02, seed=None,
       silent=True, subsample=1)

xgb.fit(X_train, y_train)

y_pred = xgb.predict_proba(X_test)

roc = roc_auc_score(y_test, y_pred[:,1])
roc

0.788228486394558

In [10]:
import xgboost
model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 

In [12]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [16]:
roc = roc_auc_score(y_test, y_pred)
roc

0.7416728174603175

In [ ]:
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',  # 多分类的问题
    'num_class': 10,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 12,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,              # 随机采样训练样本
    'colsample_bytree': 0.7,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 1,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.007,                  # 如同学习率
    'seed': 1000,
    'nthread': 4,                  # cpu 线程数
}

In [ ]:
xgboost.train(params,dtrain,num_boost_round=10,evals=(),obj=None,feval=None,maximize=False,early_stopping_rounds=None,
evals_result=None,verbose_eval=True,learning_rates=None,xgb_model=None)

In [ ]:
watchlist = [(dtrain,'train'),(dval,'val')]
model = xgb.train(params,dtrain,num_boost_round=100000,evals = watchlist)

In [ ]:
num_round = 10
bst = xgb.train( plst, dtrain, num_round, evallist )

In [ ]:
# X_test类型可以是二维List，也可以是numpy的数组
dtest = DMatrix(X_test)
ans = model.predict(dtest)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier

In [ ]:
clf_GB = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, random_state=42)
clf_GB.fit(X_train, y_train)
y_pred_GB = clf_GB.predict(X_val)

## SVM

In [ ]:
'''
g, c = 1, 10
gamma, C, roc = [], [], []
while g < 50:
    while c < 50:
        clf = svm.SVC(kernel='rbf', random_state=1, gamma=g, C=c)  
        clf.fit(X_train, y_train)
        roc.append(roc_auc_score(y_test, clf.predict(X_test)))
        gamma.append(g)
        C.append(c)
        c += 10
    c=10
    g+=10   
'''

In [27]:
from sklearn import svm
clf = svm.SVC(kernel='rbf', random_state=1, gamma=10, C=30)  
clf.fit(X_train, y_train)
roc = roc_auc_score(y_test, clf.predict(X_test_std))
roc

0.5

## Random Forest

In [23]:
from sklearn.ensemble import RandomForestClassifier

clf_RF = RandomForestClassifier(n_estimators=300, criterion='gini', max_depth=15,
                                min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                                max_features='auto', max_leaf_nodes=None, min_impurity_split=1e-07, 
                                bootstrap=True, oob_score=False, n_jobs=1, 
                                random_state=1, verbose=2, warm_start=False, class_weight='balanced')
clf_RF.fit(X_train, y_train)
y_pred_RF = clf_RF.predict(X_test)
roc = roc_auc_score(y_test, y_pred_RF)
roc

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\doggy\Anaconda3\lib\site-packages\sklearn\tree\tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
C:\Users\doggy\Anaconda3\lib\site-packages\sklearn\tree\tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)


building tree 1 of 300
building tree 2 of 300
building tree 3 of 300
building tree 4 of 300


C:\Users\doggy\Anaconda3\lib\site-packages\sklearn\tree\tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\doggy\Anaconda3\lib\site-packages\sklearn\tree\tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)


building tree 5 of 300
building tree 6 of 300


C:\Users\doggy\Anaconda3\lib\site-packages\sklearn\tree\tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\doggy\Anaconda3\lib\site-packages\sklearn\tree\tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)


...

building tree 298 of 300
building tree 299 of 300


C:\Users\doggy\Anaconda3\lib\site-packages\sklearn\tree\tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
C:\Users\doggy\Anaconda3\lib\site-packages\sklearn\tree\tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)


building tree 300 of 300


C:\Users\doggy\Anaconda3\lib\site-packages\sklearn\tree\tree.py:297: DeprecationWarning: The min_impurity_split parameter is deprecated. Its default value will change from 1e-7 to 0 in version 0.23, and it will be removed in 0.25. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   46.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    1.9s finished


0.555952380952381

In [24]:
roc

0.555952380952381

## Predict Test Data

In [ ]:
stdc = StandardScaler()
X,y = data_train.drop('Y1',axis=1), data_train['Y1']
X_train_std = stdc.fit_transform(X)
X_test_std = stdc.fit_transform(data_test)

In [ ]:
clf = svm.SVC(kernel='rbf', random_state=1, gamma=1, C=10)  
clf.fit(X_train_std, y)
y_pred = clf.predict(X_test_std)

#Submit Prediction
submit = pd.DataFrame({'CUS_ID':submit_test['CUS_ID'],'Ypred':y_pred})
submit.to_csv('SVM_reprocess_1.csv',index=False)